# Tutorial de uso para crear scripts y notebooks de python (VQE)

La idea de este tutorial es indicar el flujo de trabajo para poder usar las funcionalidad definidas. Todo lo que se muestra a continuacion puede ser usado para scripts o para trabajar en jupyters.

Hecho por: Javier Norambuena Leiva

In [1]:
import quantumsim as qs
import numpy as np

## Definir sistema

El primer paso es definir el sistema a trabajar ....

## Definir ansatz

## Definir optimizador

# Tutorial de uso para crear scripts y notebooks para ansatz adaptativo

In [ ]:
import quantumsim as qs
import numpy as np

## Construir el circuito minimo

De momento solo se considera para los modelos de Fermi-Hubbard y el de moleculas, no se ha implementado ansatz adaptativo para el modelo de Heisenberg.

Para este ejemplo, mostraremos el como usarlo en estructuras moleculas, considerar que para el modelo de Fermi-Hubbard se usan parametros distintos.

In [ ]:
symbols = ["N", "C"]
coordinates = np.array([0.0, 0.0, 2.5, 0.0, 0.0, 0.0])
params = {
    'mapping': "jordan_wigner",
    'charge': 0, 
    'mult': 1,
    'basis': 'sto-3g',
    'method': 'dhf',
}

system = qs.adap_molecular(symbols, coordinates, params)

In [ ]:
ansatz_params = {
            "repetitions": 1,
            "base": "default.qubit",
            "interface": "autograd",
            "electrons": 13,
        }

system.set_device( ansatz_params )
system.set_state( ansatz_params["electrons"] )
system.set_node( ansatz_params )

In [ ]:
minimizate_params = {
    "maxiter": 20,
    "qubits": system.qubits,
    "electrons": 13,
    "theta":["generic", 0.3]}

optimizer = qs.adap_optimizer(minimizate_params)
minsingles, mindoubles = optimizer.MinimumCircuit(system.node)

Lo que obtenemos de este proceso es un conjunto de operadores de una y dos excitaciones los cuales deben ser usados para construir el ansatz final.

Para usar estos operadores, se tiene que usar el ansatz ''custom'', el cual permite definir una cantidad arbitraria de estos operadores para construir el ansatz.

In [ ]:
system = qs.vqe_molecular(symbols, coordinates, params)

In [ ]:
ansatz_params = {
    "repetitions": 1,
    "base": "default.qubit",
    "interface": "autograd",
    "electrons": 13,
    "qubits": system.qubits,
}

ansazt = qs.custom_ansatz()
ansazt.set_device( ansatz_params )
ansazt.set_node( ansatz_params )
ansazt.set_state( ansatz_params["electrons"] )
ansazt.set_gates( singles=minsingles, doubles=mindoubles )

In [ ]:
minimizate_params = {
    "maxiter": 100,
    "tol": 1e-6,
    "number":  len(minsingles)+len(mindoubles),
    "theta":["generic", 0.25]}

system.node = ansazt.node
optimizer = qs.gradiend_optimizer(minimizate_params)
energy1, optimum = optimizer.VQE( system.cost_function )